# **Load, Display, and Save Image**

## Load Image

In [1]:
# I have installed OpenCV library and CUDA in this device
import numpy as np
import cv2
from cv2 import cuda

In [2]:
# Showing information of OpenCV library used
# Since I have installed CUDA, it should appear in this information
print(cv2.getBuildInformation())


General configuration for OpenCV 4.10.0 =====================================
  Version control:               unknown

  Extra modules:
    Location (extra):            C:/OpenCV/opencv_contrib-4.10.0/modules
    Version control (extra):     unknown

  Platform:
    Timestamp:                   2024-08-22T04:50:02Z
    Host:                        Windows 10.0.19045 AMD64
    CMake:                       3.30.2
    CMake generator:             Visual Studio 17 2022
    CMake build tool:            C:/Program Files/Microsoft Visual Studio/2022/Community/MSBuild/Current/Bin/amd64/MSBuild.exe
    MSVC:                        1941
    Configuration:               Release

  CPU/HW features:
    Baseline:                    SSE SSE2 SSE3
      requested:                 SSE3
    Dispatched code generation:  SSE4_1 SSE4_2 FP16 AVX AVX2 AVX512_SKX
      requested:                 SSE4_1 SSE4_2 AVX FP16 AVX2 AVX512_SKX
      SSE4_1 (18 files):         + SSSE3 SSE4_1
      SSE4_2 (2 files):  

In [3]:
# Read or load the image
img_bird = cv2.imread("Input/bird.jpg")

In [4]:
# Print the image as 3 dimentional matrix
print(img_bird)

[[[ 88 188 253]
  [ 88 188 253]
  [ 88 188 253]
  ...
  [ 45 137 214]
  [ 45 137 214]
  [ 44 136 213]]

 [[ 88 188 253]
  [ 88 188 253]
  [ 88 188 253]
  ...
  [ 45 137 214]
  [ 45 137 214]
  [ 45 137 214]]

 [[ 88 188 253]
  [ 88 188 253]
  [ 88 188 253]
  ...
  [ 46 138 215]
  [ 45 137 214]
  [ 45 137 214]]

 ...

 [[ 73 170 226]
  [ 72 169 225]
  [ 70 170 225]
  ...
  [  1  36  50]
  [ 29  77  95]
  [ 67 120 140]]

 [[ 73 170 226]
  [ 73 170 226]
  [ 70 170 225]
  ...
  [  0  32  46]
  [ 11  61  81]
  [ 29  83 106]]

 [[ 75 172 228]
  [ 73 170 226]
  [ 72 170 224]
  ...
  [  0  33  50]
  [  2  51  77]
  [ 94 148 178]]]


In [5]:
# Print image shape
# This shows the resolution and number of image channels
print(img_bird.shape)

(681, 640, 3)


(681, 640, 3) indicates (height (H), width (W), number of channels (C)). H x W is usually called image resolution, while C = 3 represents RGB, but in OpenCV the loaded image is in **BGR** format.

In [6]:
# Showing the image
cv2.imshow("Bird", img_bird)
cv2.waitKey(5000) # Show the image for 5000 ms or 5 s and then close it
cv2.destroyAllWindows() # This function must be present if using the cv2.waitKey function to display a temporary image

In [13]:
# Showing the image
cv2.imshow("Bird", img_bird)
cv2.waitKey(0) # Show the image until you close escape/exit button
cv2.destroyAllWindows() # This function must be present if using the cv2.waitKey function to display a temporary image

## Save Image

In [8]:
# save image with different extension (jpg to png)
cv2.imwrite("Output/bird.png", img_bird)

True

## Try using CUDA and Compare Processing Speed with CPU

In [11]:
# Load and convert image to grayscale using CUDA

# Read image
img_flm = cv2.imread("Input/mugiwara pirate.jpg")

# Upload image to GPU memory
gpu_img = cv2.cuda_GpuMat()
gpu_img.upload(img_flm)

# Use CUDA to convert the image to grayscale
gpu_gray = cuda.cvtColor(gpu_img, cv2.COLOR_BGR2GRAY)

# Download the result back to CPU
result = gpu_gray.download()

# Show the result in separate window
cv2.imshow("Grayscale Flamengo", result)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [12]:
# Processing Speed Comparison between CPU and GPU
import time

# Read image
img = cv2.imread("Input/factory.JPG")

# Measure CPU time
start_cpu = time.time()
gray_cpu = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
end_cpu = time.time()
print(f"CPU time to convert BGR to grayscale: {end_cpu - start_cpu:.4f} s")

# -------------------------------------------------------------------------

# Upload image to GPU first
gpu_img = cv2.cuda_GpuMat()
gpu_img.upload(img)

# Measure GPU time
start_cuda = time.time()
gpu_gray = cuda.cvtColor(gpu_img, cv2.COLOR_BGR2GRAY)
end_cuda = time.time()
result = gpu_gray.download()
print(f"CUDA time to convert BGR to grayscale: {end_cuda - start_cuda:.4f} s")

CPU time to convert BGR to grayscale: 0.0070 s
CUDA time to convert BGR to grayscale: 0.0133 s


**Why is GPU slower than CPU in this task?**

The GPU is usually faster for parallel tasks, but there are a few reasons why it's slower than the CPU for converting an image from BGR to grayscale:

1. **Data Transfer Overhead**: Moving data from the computer’s main memory (RAM) to the GPU's memory (VRAM) takes extra time. For small tasks like BGR to grayscale conversion, this transfer time is significant, even though the processing itself is fast.
   - **Solution**: Use the GPU for bigger or more complex tasks where data transfer is balanced by faster processing.

2. **Small Data Size**: Converting BGR to grayscale is a simple task, and it doesn’t take advantage of the GPU's ability to do many operations in parallel. For small tasks, the CPU is often faster.
   - **Solution**: Use GPU for larger tasks like batch processing, deep learning, or image filtering.

3. **GPU Initialization Time**: The first time you use the GPU, there is some extra time to set up and allocate memory. This setup time makes small tasks slower.
   - **Solution**: Run multiple GPU tasks together, so the setup time is less noticeable.

4. **Parallelism vs Task Size**: The GPU is designed for parallel processing. If the task isn’t large enough to benefit from this, the CPU’s high clock speed is more efficient.
   - **Solution**: Use the GPU for tasks that need a lot of parallel processing, like image segmentation or deep learning.

5. **CUDA Overhead**: Some tasks like `cvtColor` may not be fully optimized for GPU use and could be faster on the CPU.
   - **Solution**: Try more complex operations or process multiple images at once to better utilize the GPU.

**Conclusion:**
For small tasks like BGR to grayscale conversion, the GPU can be slower due to data transfer and setup time. The GPU works best for large, parallel tasks where it can really show its power.